In [1]:
categories = ['alt.atheism', 'soc.religion.christian',
              'comp.graphics', 'sci.med']

In [9]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.cross_validation import cross_val_score
from sklearn import tree
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.metrics import accuracy_score

In [10]:
twenty_train = fetch_20newsgroups(subset='train',
    categories=categories, shuffle=True, random_state=42)

In [11]:
from sklearn.feature_extraction.text import CountVectorizer

In [12]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(twenty_train.data)
X_train_counts.shape

(2257, 35788)

In [13]:
from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)
X_train_tf.shape
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(2257, 35788)

In [15]:
clf = svm.SVC()
clf = clf.fit(X_train_tfidf, twenty_train.target)


#np.mean(predicted == diabetes_y_test)
#accuracy_score(diabetes_y_test, predicted)
#clf = tree.DecisionTreeClassifier()
#clf = clf.fit(X_train_tfidf, twenty_train.target)

In [16]:
docs_new = ['God is love', 'OpenGL on the GPU is fast']
X_new_counts = count_vect.transform(docs_new)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)


predicted = clf.predict(X_new_tfidf)

#for doc, category in zip(docs_new, predicted):
 #   print('%r => %s' % (doc, twenty_train.target_names[category]))

In [19]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', svm.SVC(C=1.0, kernel='rbf', degree=3, gamma='auto', coef0=0.0, shrinking=True, probability=False, tol=0.001, 
              cache_size=200, class_weight=None, verbose=False, max_iter=100, decision_function_shape=None, random_state=None)),])

In [20]:
text_clf = text_clf.fit(X_train_tfidf, twenty_train.target)

AttributeError: lower not found

In [60]:
import numpy as np
twenty_test = fetch_20newsgroups(subset='test',
    categories=categories, shuffle=True, random_state=42)
docs_test = twenty_test.data
predicted = text_clf.predict(docs_test)
np.mean(predicted == twenty_test.target)   #compare target with the predicted

0.70239680426098539

In [52]:
from sklearn import metrics
print(metrics.classification_report(twenty_test.target, predicted,
    target_names=twenty_test.target_names))

                        precision    recall  f1-score   support

           alt.atheism       0.64      0.66      0.65       319
         comp.graphics       0.70      0.79      0.74       389
               sci.med       0.67      0.60      0.63       396
soc.religion.christian       0.82      0.79      0.80       398

           avg / total       0.71      0.71      0.71      1502



In [53]:
from sklearn.metrics import confusion_matrix
def plot_confusion_matrix(cm, title='Confusion matrix', cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(twenty_train.target_names))
    plt.xticks(tick_marks, twenty_train.target_names, rotation=45)
    plt.yticks(tick_marks, twenty_train.target_names)
    #plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [54]:
cm = confusion_matrix(twenty_test.target, predicted,)
np.set_printoptions(precision=2)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
print('Normalized confusion matrix')
print(cm_normalized)
plt.figure()
plot_confusion_matrix(cm_normalized, title='Normalized confusion matrix')

plt.show()

Normalized confusion matrix
[[ 0.66  0.06  0.13  0.14]
 [ 0.05  0.79  0.14  0.02]
 [ 0.15  0.21  0.6   0.04]
 [ 0.09  0.07  0.05  0.79]]
